# Custom EnergyPlus Evaluator functions
This notebook shows how to write a custom function that gets applied to the EnergyPlus outputs.

In [1]:
import pandas as pd
import numpy as np

from besos.evaluator import EvaluatorEP
from besos.problem import EPProblem
from besos import eppy_funcs as ef
from besos.parameters import RangeParameter, CategoryParameter, Parameter, FieldSelector
from besos.objectives import MeterReader, time_series_values

### Set up an EnergyPlus Evaluator
Here is a standard EnergyPlus Evaluator for editing the lighting power density for the default building and getting the electricity demand time series.  
By default, we get the sum of the variable we specify.

In [2]:
building = ef.get_building()
EPparameters = [Parameter(FieldSelector('Lights', '*', 'Watts per Zone Floor Area'),
                value_descriptor=RangeParameter(8, 12),
                name='Lights Watts/Area')]
EPobjectives = MeterReader('Electricity:Facility')
problem = EPProblem(EPparameters,EPobjectives)
evaluator = EvaluatorEP(problem,building)
result = evaluator([8])
result

(1672453961.7713666,)

### Time series data

Sometimes we want to get back an entire time series, for example for linking to the Energy Hub model (see [here](../EnergyHub/EPtoEHEvaluators.ipynb)).  
For that we can specify `func=time_series_values`, which comes as part of besos.objectives.

In [3]:
EPobjectives = MeterReader('Electricity:Facility',func=time_series_values)
problem = EPProblem(EPparameters,EPobjectives)
evaluator = EvaluatorEP(problem,building)
result = evaluator([8])
result

(0     5.041708e+07
 1     5.142561e+07
 2     5.193728e+07
 3     5.111495e+07
 4     4.728000e+07
 5     4.371469e+07
 6     5.956208e+07
 7     5.870644e+07
 8     6.031615e+07
 9     6.127009e+07
 10    6.202866e+07
 11    6.294521e+07
 12    6.369996e+07
 13    6.431286e+07
 14    6.507253e+07
 15    6.522390e+07
 16    6.471212e+07
 17    6.365803e+07
 18    6.167204e+07
 19    6.648230e+07
 20    6.682431e+07
 21    6.601229e+07
 22    4.320751e+07
 23    4.910526e+07
 24    1.431370e+07
 25    1.310912e+07
 26    1.431370e+07
 27    1.310912e+07
 28    1.431370e+07
 29    1.310912e+07
 30    1.551827e+07
 31    8.609274e+06
 32    7.227474e+06
 33    7.227474e+06
 34    7.227474e+06
 35    7.227474e+06
 36    7.227474e+06
 37    7.227474e+06
 38    7.227474e+06
 39    7.227474e+06
 40    9.991074e+06
 41    1.551827e+07
 42    1.407278e+07
 43    1.310912e+07
 44    1.431370e+07
 45    1.310912e+07
 46    1.431370e+07
 47    1.310912e+07
 Name: Value, dtype: float64,)

### Define a custom function
Now we will define the function that we will apply to the EnergyPlus output data.  
The function `highloadhoursfunc` counts the hours in which the load is above a threshold.

In [4]:
def highloadhoursfunc(result):
    threshold = 1e7
    highloadhours = sum(result.data['Value'] >= threshold)
    return highloadhours

By specifying `func=highloadhoursfunc`, the result is the number of hours in which `Electricity:Facility` is above the threshold.

In [5]:
EPobjectives = MeterReader('Electricity:Facility',func=highloadhoursfunc)
problem = EPProblem(EPparameters,EPobjectives)
evaluator = EvaluatorEP(problem,building)
result = evaluator([8])
result

(38,)